<a href="https://colab.research.google.com/github/eyurekli/shiftkey-mental-2025/blob/main/TextToSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations/Setup


In [ ]:
# install openai's whisper
#!pip install git+https://github.com/openai/whisper.git

# update the packages
#!sudo apt update && sudo apt install ffmpeg

!pip install SpeechRecognition pydub

%cd ..

/


In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import os

# Check if the directory exists before cloning the repository
if not os.path.exists("/content/Kokoro-82M"):
    # Install git LFS and clone the repository if the directory doesn't exist
    !git lfs install
    !git clone https://huggingface.co/hexgrad/Kokoro-82M
    %cd Kokoro-82M
    !apt-get -qq -y install espeak-ng > /dev/null 2>&1
    !pip install -q phonemizer torch transformers scipy munch
    %cd ..
else:
    print("Directory 'Kokoro-82M' already exists. Skipping cloning process.")


Git LFS initialized.
fatal: destination path 'Kokoro-82M' already exists and is not an empty directory.
/Kokoro-82M
/


In [ ]:
import torch; torch.cuda.is_available()

True

In [ ]:
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import display, Javascript, HTML

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = (resolve) => new Promise(async resolve => {
  let chunks = [];
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);

  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();

  const stopRecording = () => {
    recorder.stop();
    stream.getTracks().forEach(track => track.stop());  // Stop the media tracks
  }

  // Add a stop button for the user to click
  const stopButton = document.createElement('button');
  stopButton.innerText = "Stop Recording";
  stopButton.onclick = stopRecording;
  document.body.appendChild(stopButton);

  recorder.onstop = async () => {
    const blob = new Blob(chunks);
    const text = await b2text(blob);
    resolve(text);
  }
})
"""

def record():
    print("")
    print("Speak Now... Click 'Stop Recording' to end.")
    display(Javascript(RECORD))

    # Wait for the JS to return the audio data
    s = output.eval_js('record()')

    print("Done Recording !")
    b = b64decode(s.split(',')[1])
    return b  # Return byte stream of audio

# Main Program

In [31]:

import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(
        temperature=0.7,
    ))
chat = model.start_chat(history=[])

In [32]:
audio_data = record()

# Save the audio as an MP3 file
speech_file_path = '/content/speechTest.WAV'
with open(speech_file_path, 'wb') as f:
    f.write(audio_data)

print(f"Speech saved to {speech_file_path}")


Speak Now... Click 'Stop Recording' to end.


<IPython.core.display.Javascript object>

Done Recording !
Speech saved to /content/speechTest.WAV


In [33]:

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install pydub

import speech_recognition as sr
from pydub import AudioSegment

filename = "/content/speechTest.WAV"

# Convert the audio to WAV format using pydub
audio = AudioSegment.from_file(filename)  # Assuming the original format is WebM
audio.export(filename, format="wav") # Save the recording as a wav file

# Initialize the recognizer
r = sr.Recognizer()

# Open the converted WAV file
with sr.AudioFile(filename) as source:
    # Listen for the data (load audio to memory)
    audio_data = r.record(source)
    # Recognize (convert from speech to text)
    text = r.recognize_google(audio_data)

# Specify the file name
file_name = "/content/speechTest.txt"

# Write the string to a .txt file
with open(file_name, "w") as file:
    file.write(text)

In [34]:
prompt = """This message contains a brief description of an individual experiencing a mental health issue and requesting assistance.
Given the user's description, respond in a conversational and concise manners, offering support and assistance in a compassionate way. INCLUDE NO ASTERISKS IN THE RESPONSE. Keep this to no more than 8 sentences.
"""


with open("/content/speechTest.txt", "r") as file:
     sample_context = file.read()

response = chat.send_message([prompt,sample_context])



# Specify the file name
file_name = "/content/tts.txt"

# Write the string to a .txt file
with open(file_name, "w") as file:
    file.write(response.text)



In [35]:
%cd /Kokoro-82M/

from models import build_model
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL = build_model('kokoro-v0_19.pth', device)
VOICE_NAME = [
    'af', # Default voice is a 50-50 mix of Bella & Sarah
    'af_bella', 'af_sarah', 'am_adam', 'am_michael',
    'bf_emma', 'bf_isabella', 'bm_george', 'bm_lewis',
    'af_nicole', 'af_sky',
][1]
VOICEPACK = torch.load(f'voices/{VOICE_NAME}.pt', weights_only=True).to(device)
print(f'Loaded voice: {VOICE_NAME}')


/Kokoro-82M


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Loaded voice: af_bella


In [36]:
with open("/content/tts.txt", "r") as file:
     text = file.read()


In [37]:
from kokoro import generate

# Language is determined by the first letter of the VOICE_NAME:
# 🇺🇸 'a' => American English => en-us
# 🇬🇧 'b' => British English => en-gb


audio = []
for chunk in text.split("."):
    print(chunk)
    if len(chunk) < 2:
        # a try except block for non verbalizable text is probably better than this hack
        continue
    snippet, _ = generate(MODEL, chunk, VOICEPACK, lang=VOICE_NAME[0])
    audio.extend(snippet)


I hear you're going through a tough time with social anxiety
  It takes courage to reach out, and that's a great first step
  Let's work on a plan


  How about starting with short, daily walks in a less crowded area for the first week?  Then, next week, we can try adding in small interactions, like saying hello to a shop assistant
  We can gradually increase the challenge each day
 Remember, progress, not perfection, is key
  Don't hesitate to reach out if you need to adjust the plan or just want to talk
  We can get through this together




In [38]:
from IPython.display import display, Audio
display(Audio(data=audio, rate=24000, autoplay=True))